### Step 1: Data generation and training

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
# Define a custom dataset class
class ImageDataset(Dataset):
    def __init__(self, input_images, output_images, labels, transform=None):
        self.input_images = input_images
        self.output_images = output_images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.output_images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Function to load images from a directory
def load_images_from_directory(directory, target_size=(256, 256)):
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])
    images = []
    for filename in sorted(os.listdir(directory)):
        img_path = os.path.join(directory, filename)
        img = datasets.folder.default_loader(img_path)
        img = transform(img)
        images.append(img)
    return torch.stack(images)

# Directories
input_dir = '/path/to/dataset/input_images'
output_dir1 = '/path/to/dataset/output_model1'
output_dir2 = '/path/to/dataset/output_model2'
output_dir3 = '/path/to/dataset/output_model3'
output_dir4 = '/path/to/dataset/output_model4'

# Load images
input_images = load_images_from_directory(input_dir)
output_images_model1 = load_images_from_directory(output_dir1)
output_images_model2 = load_images_from_directory(output_dir2)
output_images_model3 = load_images_from_directory(output_dir3)
output_images_model4 = load_images_from_directory(output_dir4)

# Create dataset and labels
dataset = []
labels = []
for i in range(len(input_images)):
    dataset.append(output_images_model1[i])
    labels.append(0)  # Label for model 1
    dataset.append(output_images_model2[i])
    labels.append(1)  # Label for model 2
    dataset.append(output_images_model3[i])
    labels.append(2)  # Label for model 3
    dataset.append(output_images_model4[i])
    labels.append(3)  # Label for model 4

# Convert dataset and labels to tensors
dataset = torch.stack(dataset)
labels = torch.tensor(labels)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

# Create DataLoader objects
batch_size = 32
train_dataset = ImageDataset(input_images, X_train, y_train, transform=transforms.ToTensor())
test_dataset = ImageDataset(input_images, X_test, y_test, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define the CNN model
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  # Adjust according to the input image size
        self.fc2 = nn.Linear(128, 4)  # 4 classes for 4 models
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)  # Adjust according to the input image size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

### Step 2: Auto-encoder training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Define the AutoEncoder model
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the autoencoder model
autoencoder = AutoEncoder()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [ ]:
# Define a custom dataset class for the autoencoder
class AutoEncoderDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        return image, image

# Create DataLoader objects for the autoencoder training
transform = transforms.ToTensor()
autoencoder_dataset = AutoEncoderDataset(output_images_model1 + output_images_model2 + output_images_model3 + output_images_model4, transform=transform)
autoencoder_loader = DataLoader(autoencoder_dataset, batch_size=32, shuffle=True)

In [ ]:
# Train the autoencoder
num_epochs = 50
for epoch in range(num_epochs):
    autoencoder.train()
    running_loss = 0.0
    for images, _ in autoencoder_loader:
        optimizer.zero_grad()
        outputs = autoencoder(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(autoencoder_loader):.4f}')

# Save the trained autoencoder model
torch.save(autoencoder.state_dict(), 'autoencoder.pth')

### Step 3: Combining latent vectors and reconstruction

In [ ]:
import torch.nn.functional as F

# Load the trained classifier model
classifier = CNNClassifier()
classifier.load_state_dict(torch.load('classifier.pth'))
classifier.eval()

# Load the trained autoencoder model
autoencoder = AutoEncoder()
autoencoder.load_state_dict(torch.load('autoencoder.pth'))
autoencoder.eval()

# Define the encoder and decoder separately for easier handling
encoder = autoencoder.encoder
decoder = autoencoder.decoder

In [ ]:
# Function to combine latent vectors with weights
def combine_latent_vectors(vectors, weights):
    combined_vector = sum(w * v for w, v in zip(weights, vectors))
    return combined_vector

# Assume you have an input image to process
input_image_path = '/path/to/input/image.png'
input_image = Image.open(input_image_path).resize((256, 256))
input_image = transforms.ToTensor()(input_image).unsqueeze(0)  # Convert to tensor and add batch dimension

# Get the output images from the four individual models
output1 = model1(input_image)
output2 = model2(input_image)
output3 = model3(input_image)
output4 = model4(input_image)

# Pass these output images through the encoder part of the auto-encoder to get four latent vectors
latent_vector1 = encoder(output1).detach()
latent_vector2 = encoder(output2).detach()
latent_vector3 = encoder(output3).detach()
latent_vector4 = encoder(output4).detach()

# Use the classifier to get the softmax weights for the four latent vectors
with torch.no_grad():
    weights = classifier(input_image).softmax(dim=1).squeeze()

# Combine these latent vectors using the weights to get a single combined latent vector
combined_vector = combine_latent_vectors([latent_vector1, latent_vector2, latent_vector3, latent_vector4], weights)

# Pass the combined latent vector through the decoder part of the auto-encoder to get the final output image
final_output_image = decoder(combined_vector.unsqueeze(0))

# Convert the output tensor to an image format and save it
final_output_image = final_output_image.squeeze().permute(1, 2, 0).numpy()
final_output_image = (final_output_image * 255).astype(np.uint8)
save_path = '/path/to/output/image.png'
Image.fromarray(final_output_image).save(save_path)